# Genomics Dashboard Demo: Flask + React + Jupyter

This notebook demonstrates a simple dashboard for genomics pipeline data using a Flask backend and a React frontend. The backend serves mock genomics data, and the frontend fetches and displays it interactively.

## Outline

1. **Flask Backend**: Serves genomics pipeline data via REST API.
2. **React Frontend**: Fetches and visualizes pipeline data.
3. **Integration**: Run Flask server, launch React app, and interact in the notebook.
4. **Visualization**: Display input, output, and pipeline status.

---

## 1. Flask Backend: Setup & API

The backend will serve mock genomics pipeline data (inputs, outputs, status) for demonstration.

In [ ]:
# Flask backend for genomics pipeline data
from flask import Flask, jsonify
from threading import Thread
import time

app = Flask(__name__)

# Mock genomics pipeline data
data = {
    "inputs": [
        {"filename": "sample1.fastq", "type": "FASTQ", "size": "1.2GB"},
        {"filename": "sample2.bam", "type": "BAM", "size": "2.5GB"}
    ],
    "outputs": [
        {"filename": "sample1.vcf", "type": "VCF", "size": "120MB"},
        {"filename": "sample2.cram", "type": "CRAM", "size": "900MB"}
    ],
    "pipeline_status": [
        {"stage": "Upload", "status": "Complete"},
        {"stage": "QC", "status": "Complete"},
        {"stage": "Alignment", "status": "Running"},
        {"stage": "Variant Calling", "status": "Pending"}
    ]
}

@app.route('/api/pipeline-data')
def get_pipeline_data():
    return jsonify(data)

def run_flask():
    app.run(port=5001)

# Start Flask in a background thread (for notebook demo)
flask_thread = Thread(target=run_flask, daemon=True)
flask_thread.start()
# Wait a moment for the server to start
print("Flask backend running on http://localhost:5001/api/pipeline-data")
time.sleep(2)

---

## 2. React Frontend: Fetch & Visualize Data

The frontend will be a minimal React app (served via a cell using `%%writefile` and `npx serve`) that fetches data from the Flask backend and displays pipeline info.

In [ ]:
# Write minimal React app to fetch and display pipeline data
react_code = '''
import React, { useEffect, useState } from 'react';

function App() {
  const [data, setData] = useState(null);
  useEffect(() => {
    fetch('http://localhost:5001/api/pipeline-data')
      .then(res => res.json())
      .then(setData);
  }, []);
  if (!data) return <div>Loading...</div>;
  return (
    <div style={{fontFamily: 'sans-serif', margin: 20}}>
      <h2>Genomics Pipeline Dashboard</h2>
      <h3>Inputs</h3>
      <ul>
        {data.inputs.map((f, i) => (
          <li key={i}>{f.filename} ({f.type}, {f.size})</li>
        ))}
      </ul>
      <h3>Outputs</h3>
      <ul>
        {data.outputs.map((f, i) => (
          <li key={i}>{f.filename} ({f.type}, {f.size})</li>
        ))}
      </ul>
      <h3>Pipeline Status</h3>
      <ol>
        {data.pipeline_status.map((s, i) => (
          <li key={i}>{s.stage}: <b>{s.status}</b></li>
        ))}
      </ol>
    </div>
  );
}
export default App;
'''

# Write React app to file (src/App.js)
import os
os.makedirs('genomics_dashboard_react/src', exist_ok=True)
with open('genomics_dashboard_react/src/App.js', 'w') as f:
    f.write(react_code)

# Write index.js and package.json
index_js = """
import React from 'react';
import { createRoot } from 'react-dom/client';
import App from './App';
const root = createRoot(document.getElementById('root'));
root.render(<App />);
"""
with open('genomics_dashboard_react/src/index.js', 'w') as f:
    f.write(index_js)

index_html = """
<!DOCTYPE html>
<html lang=\"en\">
  <head>
    <meta charset=\"UTF-8\" />
    <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\" />
    <title>Genomics Dashboard</title>
  </head>
  <body>
    <div id=\"root\"></div>
  </body>
</html>
"""
os.makedirs('genomics_dashboard_react/public', exist_ok=True)
with open('genomics_dashboard_react/public/index.html', 'w') as f:
    f.write(index_html)

package_json = '''
{
  "name": "genomics-dashboard-react",
  "version": "0.1.0",
  "private": true,
  "dependencies": {
    "react": "^18.0.0",
    "react-dom": "^18.0.0"
  },
  "scripts": {
    "start": "npx serve -s build -l 3000",
    "build": "npx babel src --out-dir build --presets=@babel/preset-react,@babel/preset-env"
  }
}
'''
with open('genomics_dashboard_react/package.json', 'w') as f:
    f.write(package_json)

# Write .babelrc for JSX support
with open('genomics_dashboard_react/.babelrc', 'w') as f:
    f.write('''{ "presets": ["@babel/preset-react", "@babel/preset-env"] }''')

print("React app files written to ./genomics_dashboard_react/")

---

## 3. Integration: Install & Run

Install dependencies and run both the Flask backend and React frontend. The React app will fetch data from the Flask API and display it in the browser.

In [ ]:
# Install React and dependencies (run once)
!cd genomics_dashboard_react && npm install --silent && npm install @babel/preset-react @babel/preset-env serve --save-dev --silent

# Build React app
!cd genomics_dashboard_react && npm run build

# Serve React app (in background, port 3000)
import subprocess
react_serve = subprocess.Popen([
    "npx", "serve", "-s", "build", "-l", "3000"
], cwd="genomics_dashboard_react")
print("React frontend running on http://localhost:3000/")

---

## 4. Visualization: Open Dashboard

Open the React dashboard in your browser to view genomics pipeline data. The dashboard fetches from the Flask backend and displays inputs, outputs, and pipeline status.

In [ ]:
# Open the dashboard in a browser (or use the VS Code Simple Browser)
import webbrowser
webbrowser.open('http://localhost:3000/')
print("Dashboard opened in your browser. If not, open http://localhost:3000/ manually.")